### Example - *pytorch_speech_features*

In [1]:
from pytorch_speech_features import mfcc
from pytorch_speech_features import delta
from pytorch_speech_features import logfbank
import scipy.io.wavfile as wav
import torch
import numpy as np
%load_ext autoreload
%autoreload 2

In [2]:
device = 'cuda' # 'cuda' / 'cpu'

In [3]:
## Read wav file and tensor conversion
(rate,sig_array) = wav.read("english.wav")
sig = torch.FloatTensor(sig_array)
sig = sig.to(device)
sig.requires_grad = True

In [4]:
## Feature extraction
mfcc_feat = mfcc(sig,rate)
inp = mfcc_feat[:100,:].unsqueeze(0).unsqueeze(0).to(torch.float32)

In [5]:
## Check equality to python_speech_features
from python_speech_features import mfcc as orgmfcc
assert np.allclose(orgmfcc(sig_array,rate)[0], mfcc(sig,rate)[0].detach().cpu().numpy(), rtol=1e-03)

In [6]:
## Model in PyTorch
class Demo_MFCC_model(torch.nn.Module):
    def __init__(self):
        super(Demo_MFCC_model, self).__init__()
        self.conv = torch.nn.Conv2d(1, 4, 5)
        self.flatten = torch.nn.Flatten()
        self.linear1 = torch.nn.Linear(3456, 200)
        self.activation = torch.nn.ReLU()
        self.linear2 = torch.nn.Linear(200, 10)
        self.softmax = torch.nn.Softmax(dim = -1)

    def forward(self, x):
        x = self.conv(x)
        x = self.flatten(x)
        x = self.linear1(x)
        x = self.activation(x)
        x = self.linear2(x)
        x = self.softmax(x)
        return x

In [7]:
## Model forward pass
model = Demo_MFCC_model()
model.to(device)
preds = model(inp)

In [8]:
## Target loss and backward pass
loss = preds[0, 5]
loss.backward()

In [9]:
## Check gradient on input wav
sig.grad

tensor([-1.6400e-07, -2.0080e-07,  5.3700e-07,  ...,  0.0000e+00,
         0.0000e+00,  0.0000e+00], device='cuda:0')